In [1]:
import numpy as np
import scipy
from qiskit.circuit import Parameter, ParameterVector
from qiskit import QuantumCircuit, Aer, assemble
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SPSA
from qiskit.primitives import Estimator
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

In [2]:
hw = 7
V0 = -5.68658111

In [3]:
def operator(pauli):
    ''' pauli : (string) eg. XXZ 
        outputs: matrix form of the pauli term'''
    ops = {'I':np.array([[1,0],[0,1]]),
           'X':np.array([[0,1],[1,0]]),
           'Y':np.array([[0,-1j],[1j,0]]),
           'Z':np.array([[1,0],[0,-1]])}
    term = ops[pauli[0]]
    for i in range(1,len(pauli)):
        term = np.kron(term,ops[pauli[i]])
    return term

In [4]:
# Getting the second quantized hamiltonian for pionless EFT potential
def delta(n,nd):
        if n==nd:
            return 1
        else:
            return 0

def T(nd,ld,l,n):
    return ((2*n + 1.5)*delta(nd,n) - np.sqrt(n**2 + 0.5*n)*delta(nd+1,n) - np.sqrt((n+1)*(n+1.5))*delta(nd-1,n))*(hw/2)

def V(nd,n):
    return V0*delta(nd,n)*delta(0,n)

def Hamiltonian(n):
    term = []
    energy = []
    for i in range(n):
        for j in range(n):
            k = T(i,0,0,j) + V(i,j)
            if k != 0:
                term.append([i,j])
                energy.append(k)
    return term, energy

In [5]:
def fermionicJW(dagger,j,n):
    a = 'I'
    for i in range(j+1,n):
        a+='I'
    ax = a[1:]+'X'
    ay = a[1:]+'Y'
    for i in range(j):
        ax+='Z'
        ay+='Z'
    term = 0.5*operator(ax)+0.5*((-1)**dagger)*(1j)*operator(ay)
    return term

def JW_Ham(n):
    H, E = Hamiltonian(n)
    ham = 0
    for i in range(len(E)):
        ham+=E[i]*(fermionicJW(1,H[i][0],n)@fermionicJW(0,H[i][1],n))
    return ham

In [6]:
# Function that changes an Occupation basis state to Parity basis state
def OccToParity(x):
    k = np.matmul(np.triu(np.ones(len(x))),np.transpose(x))
    m = []
    for i in range(len(k)):
        if k[i]%2 == 0:
            m.append(0)
        else:
            m.append(1)
    return m

# Function that changes an Occupation basis state to Barvyi-Kitaev basis state
def OccToBK(x):
    n = int(np.log2(len(x)))
    k = np.matmul(beta(n),np.transpose(x))
    m = []
    for i in range(len(k)):
        if k[i]%2 == 0:
            m.append(0)
        else:
            m.append(1)
    return m

In [7]:
def beta(n):    
    beta = [[1,1],[0,1]]
    for i in range(n-1):
        beta = np.kron(np.eye(2),beta)
        for j in range(len(beta)):
            beta[0][j] = 1
    return beta

def update(j,n):
    k = []
    mat = beta(n)
    for i in range(len(mat)):
        if mat[i][(2**n)-(j+1)] == 1:
            k.append((2**n)-(i+1))
    k.remove(j)
    return k

def parity_matrix(n):
    mat = [[1,1],[0,1]]
    for i in range(n-1):
        mat = np.kron(np.eye(2),mat)
        for j in range(int(len(mat)/2)):
            mat[j][int(len(mat)/2)] = 1
    return mat

def flip(j,n):
    k = []
    mat = np.absolute(np.linalg.inv(beta(n)))
    for i in range(len(mat)):
        if mat[(2**n-(j+1))][i] == 1:
            k.append((2**n)-(i+1))
    k.remove(j)
    return k

def parity(j,n):
    k = []
    mat = parity_matrix(n)
    for i in range(len(mat)):
        if mat[(2**n)-(j+1)][i] == 1:
            k.append((2**n)-(i+1))
    k.remove(j)
    return k

def fermionicBK(dagger,j,n):
    k = int(np.ceil(np.log2(n)))
    term1 = 'I'
    term2 = 'I'
    remainder = list(set(parity(j,k))-set(flip(j,k)))
    for i in range(1,n+1):
        if np.isin(n-i,update(j,k)):
            term1+='X'
        elif (n-i)==j:
            term1+='X'
        elif np.isin(n-i,parity(j,k)):
            term1+='Z'
        else:
            term1+='I'
    
    for i in range(1,n+1):
        if np.isin(n-i,update(j,k)):
            term2+='X'
        elif (n-i)==j:
            term2+='Y'
        elif np.isin(n-i,remainder):
            term2+='Z'
        else:
            term2+='I'
    
    return 0.5*(operator(term1[1:])+((-1)**dagger)*(1j)*operator(term2[1:]))

def BK_Ham(n):
    H, E = Hamiltonian(n)
    ham=0
    for i in range(len(E)):
        ham+=E[i]*(fermionicBK(1,H[i][0],n)@fermionicBK(0,H[i][1],n))
    return ham

In [8]:
def gray(n):
    bit = ['0','1']
    for i in range(n-1):
        bit_rev = [bit[len(bit)-(i+1)] for i in range(len(bit))]
        bit = ['0'+bit[i] for i in range(len(bit))]
        bit_rev = ['1'+bit_rev[i] for i in range(len(bit))]
        bit = bit + bit_rev
    return bit

def TS_set(i,j,n):
    g = gray(n)
    T = []
    S = []
    for k in range(n):
        if g[i][n-(k+1)] == g[j][n-(k+1)]:
            T.append(k)
        else:
            S.append(k)
    return T,S

def fermionicGC(i,j,n):
    k = int(np.ceil(np.log2(n)))
    g = gray(k)
    code1 = g[i]
    code2 = g[j]
    T, S = TS_set(i,j,k)
    P = [0.5*(operator('I') + operator('Z'))   , 0.5*(operator('I') - operator('Z'))]
    Q = [0.5*(operator('X') + 1j*operator('Y')), 0.5*(operator('X') - 1j*operator('Y'))]
    c = 1
    if i==j:
        c=0.5
    
    if np.isin(0,T):
        term1 = term2 = P[int(g[i][k-1])]
    else:
        term1 = Q[int(g[i][k-1])]
        term2 = Q[int(g[j][k-1])]
        
    for m in range(1,k):
        if np.isin(m,T):
            term1 = np.kron(P[int(g[i][k-1-m])],term1)
            term2 = np.kron(P[int(g[j][k-1-m])],term2)
        else:
            term1 = np.kron(Q[int(g[i][k-1-m])],term1)
            term2 = np.kron(Q[int(g[j][k-1-m])],term2)
    return c*(term1+term2)

def GC_Ham(n):
    k = int(np.ceil(np.log2(n)))
    H, E = Hamiltonian(n)
    num = []
    for i in range(n):
        for j in range(i+1):
            num.append([j,i])
            
    k = []
    for i in range(len(H)):
        if [H[i][0],H[i][1]] in num:
            k.append(i)
            
    ham = 0+0*1j
    for i in k:
        ham+=E[i]*fermionicGC(H[i][0],H[i][1],n)
    return ham

In [9]:
def JW_circuit(n):
    qc = QuantumCircuit(n)
    params = ParameterVector('theta',n-1)
    qc.x(n-1)
    qc.ry(params[0],n-2)
    for i in range(1,n-1):
        qc.cx(n-(i+1),n-(i))
        qc.cry(params[i],n-(i+1),n-(i+2))
    qc.cx(0,1)
    return qc

def BK_circuit(n):
    qc = QuantumCircuit(n)
    params = ParameterVector('theta',n-1)
    qc.x(n-1)
    qc.ry(params[0],n-2)
    for i in range(1,n-1):
        qc.cx(n-(i+1),n-(i))
        qc.cry(params[i],n-(i+1),n-(i+2))
    return qc

def ansatz(n,encode):
    p1 = Parameter('theta 1')
    p2 = Parameter('theta 2')
    p3 = Parameter('theta 3')
    
    if encode=="GC":
        if n==2:
            qc = QuantumCircuit(1)
            qc.ry(p1,0)
        if n==3:
            qc = QuantumCircuit(2)
            qc.ry(p1,0)
            qc.ry(-p2,1)
            qc.cx(0,1)
            qc.ry(p2,1)
        if n==4:
            qc = QuantumCircuit(2)
            qc.ry(p2,0)
            qc.ry(p1,1)
            qc.cx(0,1)
            qc.ry(p3,1)
            
    if encode=="JW":
        qc = JW_circuit(n)
    
    if encode=="BK":
        qc = BK_circuit(n)
            
    return qc

In [10]:
# Hamiltonians for EFT Potential
EFT = { 'JW' : { 2 : JW_Ham(2), 3 : JW_Ham(3) },
        'BK' : { 2 : BK_Ham(2), 3 : BK_Ham(3) },
        'GC' : { 2 : GC_Ham(2), 3 : GC_Ham(3) }}

In [11]:
import numpy as np
from qiskit import Aer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import VQE
from qiskit.aqua.components.optimizers import SPSA
from qiskit.circuit.library import TwoLocal
from qiskit.aqua.operators import MatrixOperator, WeightedPauliOperator

# Define the Hamiltonian matrix as a NumPy array
hamiltonian_matrix = np.array([[1, 0, 0, 0],
                               [0, 0, -1, 0],
                               [0, -1, 0, 0],
                               [0, 0, 0, 1]])

# Create the corresponding Qiskit operator from the Hamiltonian matrix
hamiltonian_op = MatrixOperator(matrix=hamiltonian_matrix)

# Define the initial quantum state preparation circuit
num_qubits = int(np.log2(hamiltonian_matrix.shape[0]))
initial_state = np.ones(2**num_qubits) / np.sqrt(2**num_qubits)
ansatz = TwoLocal(num_qubits, ['ry', 'rz'], 'cx', reps=1, entanglement='full', insert_barriers=True)
ansatz.compose(initial_state, front=True, inplace=True)

# Choose the classical optimizer for VQE
optimizer = SPSA(maxiter=100)

# Create the VQE algorithm instance
vqe = VQE(hamiltonian=hamiltonian_op, optimizer=optimizer, quantum_instance=QuantumInstance(Aer.get_backend('statevector_simulator')))

# Run VQE
result = vqe.run(ansatz)

# Obtain the minimum eigenvalue and corresponding eigenvector
min_eigenvalue = np.real(result['eigenvalue'])
eigenvector = result['eigenstate']

print(f"Minimum eigenvalue: {min_eigenvalue}")
print(f"Eigenvector: {eigenvector}")

ModuleNotFoundError: No module named 'qiskit.aqua'

In [12]:
!pip install qiskit-aqua

     ---------------------------------------- 2.1/2.1 MB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 133.4/133.4 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 368.2/368.2 kB 2.5 MB/s eta 0:00:00
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Installing backend dependencies: started
  Installing backend dependencies: finished with status 'error'


  error: subprocess-exited-with-error
  
  pip subprocess to install backend dependencies did not run successfully.
  exit code: 1
  
  [231 lines of output]
       ------------------------------------ 988.4/988.4 kB 659.1 kB/s eta 0:00:00
       ---------------------------------------- 7.3/7.3 MB 634.4 kB/s eta 0:00:00
    Installing build dependencies: started
    Installing build dependencies: finished with status 'done'
    Getting requirements to build wheel: started
    Getting requirements to build wheel: finished with status 'done'
    Preparing metadata (pyproject.toml): started
    Preparing metadata (pyproject.toml): finished with status 'error'
    error: subprocess-exited-with-error
  
    Preparing metadata (pyproject.toml) did not run successfully.
    exit code: 1
  
    [202 lines of output]
    setup.py:67: RuntimeWarning: NumPy 1.19.3 may not yet support Python 3.10.
      warnings.warn(
    Running from numpy source directory.
    setup.py:480: UserWarning: Unrecogn

In [11]:
CP = { 'JW' : { 2 : 7.858535*(I^I) + 0.00257*(I^Z) - 7.861105*(Z^I) - 0.37778*((X^X)+(Y^Y)),
                3 : (7.858535*(I^I^I) + 0.00257*(I^I^Z) - 7.861105*(I^Z^I) - 0.37778*((I^X^X)+(I^Y^Y)) + 15.92676*((I^I^I)-(Z^I^I)) - 3.6989*((X^Z^X)+(Y^Z^Y)) + 4.123715*((X^X^I) + (Y^Y^I))).reduce()},
       'GC' : { 2 : (7.858535*I - 7.863675*Z - 0.75556*X),
                3 : (11.892645*(I^I) - 11.895215*(I^Z) - 4.03411*(Z^I) + 4.03154*(Z^Z) - 3.6989*((X^X)-(Y^Y)) + 4.123715*((X^I) - (X^Z)) - 0.37778*((I^X)+(Z^X))) },
       'BK' : { 2 : 7.858535*(I^I) + 0.00257*(I^Z) - 7.861105*(Z^Z) - 0.37778*((I^X)-(Z^X)),
                3 : (7.858535*(I^I^I) + 0.00257*(I^I^Z) - 7.861105*(I^Z^Z) - 0.37778*((I^I^X)-(I^Z^X)) + 15.92676*((I^I^I)-(Z^I^I)) - 3.6989*((Y^Y^X)-(X^Y^Y)) + 4.123715*((X^X^I)+(Y^Y^Z))).reduce() }}

R68 = { 'JW' : 116.5294*(I^I) - 78.85401*(I^Z) - 37.67542*(Z^I) - 12.88914*((X^X)+(Y^Y)),
        'BK' : 116.5294*(I^I) - 78.85401*(I^Z) - 37.67542*(Z^Z) - 12.88914*((I^X)-(Z^X)),
        'GC' : 116.5294*I + 41.17867*Z - 25.77827*X }

In [13]:
def vqe(encode,n,H):
    circuit = ansatz(n,encode)
    vqe = VQE(estimator=Estimator(), ansatz=circuit, optimizer=SPSA(maxiter=100))
    result = vqe.compute_minimum_eigenvalue(H)
    return result.optimal_value

def ground_state(H):
    ''' H : matrix form of Hamiltonian (array)
        outputs: [ground state, ground state energy] of H'''
    H = scipy.sparse.csr_matrix(np.array(H))
    eigvals, eigvecs = scipy.sparse.linalg.eigs(H)
    return np.real(min(eigvals))

In [17]:
# EFT results
k = [[],[]]
for i in ['JW','BK','GC']:
    for j in [2,3]:
        k[j-2].append(vqe(i,j,EFT[i][j]))

k[0].append(ground_state(JW_Ham(2)))
k[1].append(ground_state(JW_Ham(3)))

results = pd.DataFrame(k)
results.columns=['JW','BK','GC','Exact']
results.index=[2,3]
print('Effective field theory Potential')
results

TypeError: Cannot use scipy.linalg.eig for sparse A with k >= N - 1. Use scipy.linalg.eig(A.toarray()) or reduce k.

In [15]:
# CP results
k = [[],[]]
for i in ['JW','BK','GC']:
    for j in [2,3]:
        k[j-2].append(vqe(i,j,CP[i][j]))

results = pd.DataFrame(k)
results.columns=['JW','BK','GC']
results.index=[2,3]
print('Central Potential')
results

Central Potential


,JW,BK,GC
2,-0.041355,-0.041355,-0.041354
3,-1.709004,-1.708289,-1.656631


In [16]:
# R68 results
k = [[]]
for i in ['JW','BK','GC']:
        k[0].append(vqe(i,2,R68[i][2]))

results = pd.DataFrame(k)
results.columns=['JW','BK','GC']
results.index=[2]
print('Reid68 Potential')
results

Reid68 Potential


,JW,BK,GC
2,-37.67542,-37.67542,-25.778269
